# 发现短文本主题模型实践
## 1.引言
许多数据分析应用都会涉及到从短文本中发现主题，比如微博、短信、日志和评论。一方面，许多进一步的分析基于这些潜在话题，比如：情感分析、文档分类；另一方面，由于短文本有着独特的特点，在使用传统的话题发现算法中应值得引起更多的关注，挑战如下：
- 短文本有着高度的灵活性，不同的表示方式有着相同的含义。比如："dollars",'$$','$'，'fee'可能有着相似的含义；由于短文本字数的限制，很难从其周围词语中获得含义。
- 不像维基百科这类长文本，短评论或者微博可能仅仅含有一个话题；乍眼一看是句子比较单一，但传统主题模型方法是假定一篇文章含有多个主题，这中稀疏表示方法引起的代价要么是计算量上的要么是性能相关的。

主题发现需要有多个步骤，包括文本的预处理、向量化、话题挖掘和最终的用关键字表示话题。其中的每一个步骤都有不同的实现方式并且不同的组合方式可能会有着不同的结果

这篇文章将展示不同主题挖掘算法中的优缺点，考虑到上面刚提到的短文本特性，本文将以实际遇到的数据出发，而不是理论介绍。

下面将基于‘toy-like’人工数据集，像使用“黑盒子”一样使用算法。所有的模型均来自scikit-learn包中。

## 2.主题发现模型
本文主要介绍三个主题模型LDA(Laten Dirichlet Allocation)、NMF(Non-Negative Matirx Factorization)和SVD(Truncated Singular Value Decomposition)。这些方法均有不同的实现方式，本文使用的是scikit-learn中的实现

除了上面提到的三个传统的主题模型，还有一些其他的一些发现文档结构的方法，本文将与k-means聚类方法进行对比，我们能够发现一些有趣的讨论，在代码的下面给出了相应的解释。

In [8]:
%matplotlib

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,NMF,LatentDirichletAllocation
from sklearn.cluster import  KMeans

Using matplotlib backend: Qt4Agg


在深入了解模型之前，我们需要先准备一些样本文本，本文人工生成了一些文本用作话题发现
- clearcut topics清晰的主题：文本中清晰的含有2个话题，"berger-lovers" and "sandwich-haters"
- unbalanced topics不平衡主题：同样包含两个话题，但话题分布不均匀。正常的场景中会出现异常的信息或评论
- semantic topics语义主题：预料中含有四个话题，分别是 "berger-lovers"， "berger-haters"，"sandwich-lovers" 和 "sandwich-haters"。此外，该数据集中还包含了两个潜在的主题 “food” 和 “feelings”。
- noisy topic噪声话题：正如上面讨论的那样，由于短文本的灵活性，可能不同的term表示相同的含义，或拼写错误。这个语料是模拟两个话题但含有错别字。这个语料的数量小于其他几个，以便于我们测试模型是怎么解决这些模糊信息

In [9]:
def gen_clearcut_topics():
    return np.repeat(["we love bergers", "we hate sandwiches"], [1000, 1000])
def gen_unbalanced_topics():
    return np.repeat(["we love bergers", "we hate sandwiches"], [10, 1000])
def gen_semantic_context_topics():
    return np.repeat(["we love bergers"
                      , "we hate bergers"
                      , "we love sandwiches"
                      , "we hate sandwiches"], 1000)
def gen_noisy_topics():
    def _random_typos(word,n):#生成错别字
        index = np.random.randint(0,len(word),n) #随机生成n个 0到len(word)的数字
        return [word[:i] + "X" +word[i+1:] for i in index]
    t1 = ["we love %s" % w for w in _random_typos("bergers",15)]
    t2 = ["we hate %s" %w for w in _random_typos("sandwiches",15)]
    return np.r_[t1,t2] #数组串联起来
sample_tests = {
    "clearcut topics":gen_clearcut_topics(),
    "unbalanced topics":gen_unbalanced_topics(),
    "semantic topics":gen_semantic_context_topics(),
    "noisy topics":gen_noisy_topics()
}

In [10]:
from collections import Counter
for desc, text in sample_tests.items():
    print desc
    print Counter(text).most_common()
    print ""

noisy topics
[('we hate sXndwiches', 4), ('we love bergeXs', 4), ('we love bergerX', 3), ('we love bXrgers', 3), ('we love Xergers', 2), ('we hate sandwiXhes', 2), ('we hate saXdwiches', 2), ('we love beXgers', 2), ('we hate Xandwiches', 2), ('we love berXers', 1), ('we hate sanXwiches', 1), ('we hate sandwichXs', 1), ('we hate sandwicheX', 1), ('we hate sandwXches', 1), ('we hate sandwicXes', 1)]

clearcut topics
[('we love bergers', 1000), ('we hate sandwiches', 1000)]

unbalanced topics
[('we hate sandwiches', 1000), ('we love bergers', 10)]

semantic topics
[('we love bergers', 1000), ('we love sandwiches', 1000), ('we hate sandwiches', 1000), ('we hate bergers', 1000)]



我们需要回顾一下并考虑怎么样才能评价一个模型的好坏程度，尽管通常依赖于实际的分析，但也有一些通用的共识，比如keywords在每个话题中应该：
- 在文档或句子中频繁出现。只覆盖到几个样本的话题不是好话题，当然除了去找outlier
- 足以去区分不同的话题。表示话题之间的相关与独立性

还有研究也给出了其他的指标，例如：
- 相同主题的相关keywords出现的频率也应该高，因为他们来自相似语境
- 话题中的keywords应该具有语义相似性。比如：“苹果”和“橘子”
让我们来对比一下这几个模型：NMF，LDA，SVD和Kmeans，他们有着同一个`find_topic`接口，将分别采用两种`vectorization`方法，例如：TF和IDF。一般来说数据集中的词语在多篇文档中都频繁出现name选择TFIDF而不是TF，有些词语频繁的在语料中出现（停用词）会对话题造成影响，然而对于短文而言TF或TFIDF并没有显著地区别，因为短文本很难出现上述情况。文档的向量化方法是热门研究方向，像基于word embedding的模型，如：w2v,d2v

主题模型选择主题词语分布中频率最高的词语最为主题的keywords，但对于SVD或Kmeans算法来说，模型得到的主题词语矩阵中包含正和负值，很难直接从中提取关键词语，为了展示，本文挑选出绝对值最大的几个词语作为关键词，并且根据正负值得情况加上相应的标签，对负向词语加上***^***,比如***^bergers***

In [11]:
def find_topic(texts,topic_model,n_topics,vec_model="tf",thr=1e-2,**kwargs):
    """
    text:array-like 文本
    topic_model: {"nmf", "svd", "lda", "kmeans"}
    n_topics:话题的数量
    vec_model:向量化表示方法{"tf", "tfidf"}
    thr:阈值
    """
    # 1.vectorization
    vectorizer = CountVectorizer() if vec_model=="tf" else TfidfVectorizer()
    text_vec = vectorizer.fit_transform(texts)
    words = np.array(vectorizer.get_feature_names())
    ## 2. topic finding
    topic_models = {"nmf": NMF, "svd": TruncatedSVD, "lda": LatentDirichletAllocation, "kmeans": KMeans}
    topicfinder = topic_models[topic_model](n_topics,**kwargs).fit(text_vec)
    topic_dists = topicfinder.components_  if topic_model is not "kmeans" else topicfinder.cluster_centers_
    topic_dists /= topic_dists.max(axis = 1).reshape((-1, 1))#转化为一个列，(1,-1)转化为一行
    
    ##keywords for topics
    ##LDA LSA在话题分布向量中会存在正和负，不利于keywords的选择，下面做出转换
    def _topic_keywords(topic_dist):
        keywords_index = np.abs(topic_dist) >= thr
        keywords_prefix = np.where(np.sign(topic_dist),"","^")[keywords_index]
        keywords = " | ".join(map(lambda x :"".join(x),zip(keywords_prefix, words[keywords_index])))
        return keywords
    
    topic_keywords = map(_topic_keywords,topic_dists)
    return "\n".join("Topic %i: %s" %(i,t) for i,t in enumerate(topic_keywords) )

### 2.1 SVD：正交分解
sklearn中的truncated SVD类似于PCA算法，它们都试图利用正交分解的方法选择出具有最大方差的变量信息
对于`clearcut-topic`语料，我们分别使用TFIDF、TF方法来向量化文本数据，并构建SVD模型，我们得到的结论如下所示。正如我们刚才提到的，SVD模型所提取的keywords包含正负向词语，为了简单起见，我们可以理解为该主题包含正向词语，不包含负向的词语

比如` "Topic 1: bergers | ^hate | love | ^sandwiches"`来说，该文本包含"love berger"不包含"hate sandwiches"

由于模型的随机性，每次运行的话题结果都会有一定的不同，在SVD的训练结果中，我们发现`Topic 3:bergers | ^hate | ^love | sandwiches`成功的提取了`food`这个主题

In [12]:
print(find_topic(sample_tests["clearcut topics"],"svd",4,vec_model="tf"))

Topic 0: bergers | hate | love | sandwiches | we
Topic 1: bergers | hate | love | sandwiches
Topic 2: bergers | hate | love | sandwiches | we
Topic 3: bergers | hate | love | sandwiches


In [13]:
print(find_topic(sample_tests["clearcut topics"],"svd",4,vec_model="tfidf"))

Topic 0: bergers | hate | love | sandwiches | we
Topic 1: bergers | hate | love | sandwiches
Topic 2: bergers | hate | love | sandwiches | we
Topic 3: bergers | hate | love | sandwiches


在上面的例子中我们设置了过多的主题数量，因为在大多数的时候骂我们无法事先知道某个文档包含多少个主题，如果令主题的个数为2，可以得到下述结果：

In [14]:
print(find_topic(sample_tests["clearcut topics"],"svd",2,vec_model="tfidf"))

Topic 0: bergers | hate | love | sandwiches | we
Topic 1: bergers | hate | love | sandwiches


当我们在解释SVD模拟的结果时，我们需要对比多个主题信息，比如上述的模型拟合结果可以解释成：数据集中文档的主要差异是文档中包含"love berger"但不包含"hate sandwiches"接下来利用SVD来拟合unbalenced topic数据集，检验处理非平衡数据集的效果。

In [15]:
print(find_topic(sample_tests["noisy topics"],"svd",2,vec_model="tfidf"))

Topic 0: bergerx | bergexs | berxers | bexgers | bxrgers | hate | love | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | sxndwiches | we | xandwiches | xergers
Topic 1: bergerx | bergexs | berxers | bexgers | bxrgers | hate | love | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | sxndwiches | we | xandwiches | xergers


从上面结果可以看出SVD无法处理噪声数据，不能从中提取出主题信息

In [16]:
print(find_topic(sample_tests["unbalanced topics"],"svd",2,vec_model="tfidf"))

Topic 0: hate | sandwiches | we
Topic 1: bergers | hate | love | sandwiches | we


### 2.2 LDA：根据词语的共现频率来提取主题
LDA是最常用的主题提取模型之一，因为该模型能够处理多种类型的文本数据，而且模型的拟合效果非常容易解释

直观上看，***LDA根据不同文档中词语的共现频率来提取文本中潜在的主题信息。另一方面，具有相同主题结构的文本之间往往非常相似***，因此我们可以根据潜在的主题空间来推断词语之间的相似性和文档之间的相似性。

LDA算法主要有两类参数：
- 每个主题中各个关键词的分布参数
- 每篇文档中各个主题的分布参数

接下来我们将看到这些参数是怎么在非平衡数据集中是怎么发现话题的，找到合适的参数，人们更多的是根据经验来选择参数

与SVD模型不同的是，LDA模型提取的主题容易解释。以clearcut-topics数据集为例，LDA模型中每个主题都有明确的关键词，它和SVD主要有两个区别：
- LDA模型中可能存在重复的主题
- 不同的主题可以共享相同的keywords，比如单词“we”在所有的主题中都有出现

此外，对LDA模型来说，采用不同的文本向量化方法`vectorization`也会得到不同的结果

In [44]:
print(find_topic(sample_tests["clearcut topics"],"lda",4,vec_model="tf"))

Topic 0: bergers | love | we
Topic 1: hate | sandwiches | we
Topic 2: bergers | love | we
Topic 3: bergers | love | we


In [46]:
print(find_topic(sample_tests["clearcut topics"],"lda",4,vec_model="tfidf"))

Topic 0: bergers | love | we
Topic 1: bergers | love | we
Topic 2: hate | sandwiches | we
Topic 3: bergers | love | we


本文将介绍如何调整参数来处理unbalanced语料，在sklearn实现方法中参数是`topic_word_prior`和` doc_topic_prior`分别控制LDA模型的两类参数，其中`topic_word_prior`的默认值为$\frac{1}{n\_topics} $，假定主题分布服从均匀分布。

In [19]:
print(find_topic(sample_tests["unbalanced topics"], "lda", 4, vec_model="tf"))

Topic 0: bergers | hate | love | sandwiches | we
Topic 1: hate | sandwiches | we
Topic 2: hate | sandwiches | we
Topic 3: hate | sandwiches | we


***使用一个较小的`topic_word_prior`值将能找到更细粒度的主题，因为每个主题中都会选择更少的词语。***

In [21]:
print(find_topic(sample_tests["unbalanced topics"], "lda", 4, vec_model="tf", topic_word_prior=1e-5))

Topic 0: hate | sandwiches | we
Topic 1: hate | sandwiches | we
Topic 2: hate | sandwiches | we
Topic 3: bergers | hate | love | sandwiches | we


LDA模型同样***无法很好地处理noisy topics数据集***，从下面结果中可以看到LDA模型提取的主题也相当模糊

In [23]:
print find_topic(sample_tests["noisy topics"],"lda",3, vec_model = "tfidf")

Topic 0: bergerx | bergexs | berxers | bexgers | bxrgers | hate | love | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | sxndwiches | we | xandwiches | xergers
Topic 1: bergerx | bergexs | berxers | bexgers | bxrgers | hate | love | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | sxndwiches | we | xandwiches | xergers
Topic 2: bergerx | bergexs | berxers | bexgers | bxrgers | hate | love | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | sxndwiches | we | xandwiches | xergers


总结：

- 通过LDA生成的模型容易被人理解，因为它是按照词频共现
- （一篇文章中肯能出现多个主题）有些时候这些主题并不能区分不同的doc，-有时使用话题来表示一个文档可能对此有帮助
- 与SVD的结果不同
- 利用LDA生成的topic来表示一个DOC能够应用在其他方面，比如doc classification
- 更好的理解数据有助于生成较好的LDA，但最初我们可能并没有领域知识，并且LDA的运行成本较高，因此我们使用其他方法来理解数据。

### 2.3 类LDA的分解方法
NMF可以视为LDA模型的特例，从理论上来说，这两个模型之间的联系非常复杂。但是在实际引用中，NMF经常被视为参数固定且可以获得稀疏解的LDA模型。虽然NMF模型的灵活性不如LDA模型，但是该模型可以很好的处理短文本数据集。

另一方面，NMF最大的缺点***是拟合结果的不一致***-当我们设置`最大的主题个数`时NMF拟合的结果非常糟糕。相比之下，LDA模型的拟合结果更为稳健。

首先我们来看下NMF模型不一致的拟合结果。对于clearcut topic数据集说，当我们设置5个主题的时候（真是主题个数为2），其结果和真实结果非常相似

In [28]:
print find_topic(sample_tests["clearcut topics"],"nmf",5 ,vec_model="tf")

Topic 0: bergers | love | we
Topic 1: hate | sandwiches | we
Topic 2: hate | sandwiches | we
Topic 3: bergers | love | we
Topic 4: hate | sandwiches | we


但是当我们增加主题个数到25时，远大于真实的2，NMF将会出现一些离奇的结果

In [30]:
print(find_topic(sample_tests["clearcut topics"], "nmf", 25, vec_model="tf"))

Topic 0: hate | sandwiches | we
Topic 1: bergers | love | we
Topic 2: love | we
Topic 3: hate | sandwiches | we
Topic 4: bergers | love | we
Topic 5: hate | sandwiches | we
Topic 6: we
Topic 7: bergers | love | we
Topic 8: love | we
Topic 9: hate | sandwiches | we
Topic 10: sandwiches
Topic 11: love | we
Topic 12: bergers | love | we
Topic 13: bergers | we
Topic 14: hate | sandwiches | we
Topic 15: hate | sandwiches | we
Topic 16: love | we
Topic 17: hate | we
Topic 18: hate | sandwiches | we
Topic 19: we
Topic 20: hate | we
Topic 21: hate | sandwiches | we
Topic 22: bergers | love | we
Topic 23: hate | sandwiches
Topic 24: hate | sandwiches | we


相比之下，LDA模型的结果更为的稳健（consistent）

In [32]:
print(find_topic(sample_tests["clearcut topics"], "lda", 25, vec_model="tf"))

Topic 0: bergers | love | we
Topic 1: bergers | love | we
Topic 2: bergers | hate | love | sandwiches | we
Topic 3: bergers | hate | love | sandwiches | we
Topic 4: bergers | hate | love | sandwiches | we
Topic 5: bergers | love | we
Topic 6: bergers | love | we
Topic 7: bergers | love | we
Topic 8: bergers | hate | love | sandwiches | we
Topic 9: bergers | love | we
Topic 10: bergers | love | we
Topic 11: hate | sandwiches | we
Topic 12: bergers | love | we
Topic 13: bergers | love | we
Topic 14: bergers | love | we
Topic 15: bergers | love | we
Topic 16: bergers | love | we
Topic 17: bergers | love | we
Topic 18: bergers | love | we
Topic 19: bergers | hate | love | sandwiches | we
Topic 20: bergers | love | we
Topic 21: bergers | love | we
Topic 22: bergers | love | we
Topic 23: bergers | love | we
Topic 24: bergers | love | we


对于非平衡数据集，设置合适的主题个数，NMF可以更好的主题信息

In [34]:
print(find_topic(sample_tests["unbalanced topics"], "nmf", 5, vec_model="tfidf"))

Topic 0: hate | we
Topic 1: hate | sandwiches | we
Topic 2: hate | sandwiches | we
Topic 3: bergers | love | we
Topic 4: hate | sandwiches | we


指的注意的是，NMF是本文中提到的四个模型中唯一一个能够较好的处理noisy topic的数据模型

In [36]:
print find_topic(sample_tests["noisy topics"],"nmf",5, vec_model = "tfidf",)

Topic 0: berxers | bexgers | bxrgers | love | we | xergers
Topic 1: bexgers | hate | sandwichex | sandwichxs | sandwicxes | sandwixhes | sandwxches | sanxwiches | saxdwiches | we | xandwiches | xergers
Topic 2: bergexs | berxers | bexgers | love | we | xergers
Topic 3: bergerx | berxers | bexgers | love | we | xergers
Topic 4: hate | sandwichex | sandwichxs | sandwicxes | sandwxches | sanxwiches | sxndwiches | we


总结：

- NMF在短文本中效果较好
- NMF的计算量小于LDA
- 随着话题数量的增加，NMF的结果逐渐变得不稳定

### 2.4 KMeans: 简单且强大
类似于kmeans的聚类方法能够根据文档的向量形式对其进行分组。然而通常我们不把它看做是主题发现模型，因为它很难来解释用来分组的keywods

但是，结合tfidf或tf方法，我们可以将kmeans模型的聚类中心看做一堆词语的概率组合

In [38]:
print find_topic(sample_tests["clearcut topics"],"kmeans",10, vec_model = "tf")

Topic 0: bergers | love | we
Topic 1: hate | sandwiches | we
Topic 2: hate | sandwiches | we
Topic 3: bergers | love | we
Topic 4: bergers | love | we
Topic 5: bergers | love | we
Topic 6: bergers | love | we
Topic 7: bergers | love | we
Topic 8: bergers | love | we
Topic 9: bergers | love | we


In [40]:
print find_topic(sample_tests["unbalanced topics"],"kmeans",10, vec_model = "tf",)

Topic 0: hate | sandwiches | we
Topic 1: bergers | love | we
Topic 2: hate | sandwiches | we
Topic 3: hate | sandwiches | we
Topic 4: hate | sandwiches | we
Topic 5: hate | sandwiches | we
Topic 6: hate | sandwiches | we
Topic 7: hate | sandwiches | we
Topic 8: hate | sandwiches | we
Topic 9: hate | sandwiches | we


In [42]:
print find_topic(sample_tests["noisy topics"],"kmeans",10, vec_model = "tf",)

Topic 0: hate | sandwichex | sandwichxs | sandwicxes | sandwxches | sanxwiches | we
Topic 1: bergexs | berxers | love | we
Topic 2: hate | sxndwiches | we
Topic 3: hate | saxdwiches | we
Topic 4: hate | sandwixhes | we
Topic 5: bergerx | love | we
Topic 6: love | we | xergers
Topic 7: bxrgers | love | we
Topic 8: hate | we | xandwiches
Topic 9: bexgers | love | we


总结：

像NMF一样，kmean 在不同的短文本上面具有不错的结果，并且在设置不同的topic数量的时候，结果更加的稳定。

### 2.5 寻找语义相关的主题
最后我们简单谈一下从不同角度的主题发现方法，大多数情况下我们倾向于根据文档的主题分布情况对文档进行分组，并根据关键词的分布情况来提取主题信息。

大多数研究者认为词语的语义信息是由其上下文决定的，比如‘love’和‘hate’可以看成语义相近的词语，因为这两个词语都可以运用在相同的语境当中，事实上，词向量最重要的一个研究是如何构建词语、短语或者文档的向量形式，使得新的向量空间中仍然保留着语义信息。

找寻语义相同的词语不同于计算词语的贡献频率。从下述的结果中可以看出，大多数主题提取模型只涉及到词语的共现频率，并没有考虑到词语的语义信息，只有SVD模型简单涉及到词语的语义信息。

需要注意的是，本文采用的数据集是根据一定的规则随机生成，所以下述数据结果更多的说明不同模型之间的区别

In [44]:
print(find_topic(sample_tests["semantic topics"], "svd", 4, vec_model="tfidf"))

Topic 0: bergers | hate | love | sandwiches | we
Topic 1: bergers | hate | love | sandwiches
Topic 2: bergers | hate | love | sandwiches
Topic 3: bergers | hate | love | sandwiches | we


In [46]:
print(find_topic(sample_tests["semantic topics"], "nmf", 5, vec_model="tfidf"))

Topic 0: sandwiches | we
Topic 1: hate | we
Topic 2: love | we
Topic 3: bergers | we
Topic 4: bergers | hate | we


In [48]:
print(find_topic(sample_tests["semantic topics"], "lda", 5, vec_model="tfidf"))

Topic 0: love | sandwiches | we
Topic 1: bergers | love | sandwiches | we
Topic 2: bergers | hate | we
Topic 3: bergers | love | we
Topic 4: bergers | love | we


In [53]:
print(find_topic(sample_tests["semantic topics"], "kmeans", 5, vec_model="tfidf"))

Topic 0: bergers | love | we
Topic 1: hate | sandwiches | we
Topic 2: bergers | hate | we
Topic 3: love | sandwiches | we
Topic 4: love | sandwiches | we


## 3. 总结
- 短文本数据集具有独特的性质，在建模时候需要特别的注意
- 模型的选择依赖于主题的定义（贡献频率高或语义相似性）和主题提取的目的
- 我们可以使用kmeans或者NMF模型快速获取文档的结构信息
    - 主题中词语的分布情况
    - 文档中主题的分布情况
    - 主题个数
    - 每个主题中词语的个数
- LDA模型具有很好的灵活性，可以处理多种类型的文本数据，但是调参过程需要很好的理解数据
- SVD模型可以很好的提取出文本的主要信息，比如SVD可以很好的分析半结构化的数据